In [71]:
import pandas as pd
import datetime as dt

In [72]:
eleves = pd.read_csv('../data/DIM_ELEVE.csv', parse_dates=['DATE_NAISSANCE'])

In [73]:
print(eleves.columns)
print(eleves.shape)
print(eleves.GENRE.unique())

Index(['ID_ELEVE', 'GENRE', 'DATE_NAISSANCE', 'ID_CLASSE'], dtype='object')
(10000, 4)
['m' 'f']


In [74]:
dist_genre = eleves.groupby('GENRE').count()['ID_ELEVE']

In [75]:
reponses = pd.read_csv('../data/FACT_REPONSE.csv', parse_dates=['DATE_TIME'])

In [76]:
print(reponses.columns)
print(reponses.shape)

Index(['ID_LOG', 'KEY_REPONSE', 'ID_REPONDANT', 'DATE_TIME'], dtype='object')
(100000, 4)


In [77]:
reponses.columns

Index(['ID_LOG', 'KEY_REPONSE', 'ID_REPONDANT', 'DATE_TIME'], dtype='object')

In [78]:
merged = reponses.merge(
    eleves, left_on = "ID_REPONDANT", right_on = "ID_ELEVE"
    )

In [79]:
merged['age_group'] = pd.cut(
    # delta t annees
    (merged.DATE_TIME - merged.DATE_NAISSANCE).dt.days // 365.2425,
    bins = [0, 11, 13, 15, 17, 99],
    labels = ['0-11', '11-12', '13-14', '15-17', '>17'],
    right = False
)

In [80]:
merged.groupby("age_group").count()

/tmp/ipykernel_1628383/2799898231.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  merged.groupby("age_group").count()


,ID_LOG,KEY_REPONSE,ID_REPONDANT,DATE_TIME,ID_ELEVE,GENRE,DATE_NAISSANCE,ID_CLASSE
age_group,,,,,,,,
0-11,60173,60173,60173,60173,60173,60173,60173,60173
11-12,39827,39827,39827,39827,39827,39827,39827,39827
13-14,0,0,0,0,0,0,0,0
15-17,0,0,0,0,0,0,0,0
>17,0,0,0,0,0,0,0,0


In [81]:
n_eleves = eleves.shape[0]

In [82]:
reponses.KEY_REPONSE.value_counts()['B_2_1_d'] / n_eleves

0.0604

In [83]:
merged.loc[merged.KEY_REPONSE == 'B_2_1_d'].groupby("GENRE").count()['ID_LOG'] / dist_genre

GENRE
f    0.061860
m    0.058952
dtype: float64

In [84]:
# use distinct ID_ELEVE per age_group to get age_group distributions

In [85]:
merged.loc[merged.KEY_REPONSE == 'B_2_1_d'].groupby("age_group")['ID_LOG'].count()

/tmp/ipykernel_1628383/1569251425.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  merged.loc[merged.KEY_REPONSE == 'B_2_1_d'].groupby("age_group")['ID_LOG'].count()


age_group
0-11     353
11-12    251
13-14      0
15-17      0
>17        0
Name: ID_LOG, dtype: int64

In [86]:
merged.groupby("age_group").apply(lambda x: x.ID_REPONDANT.unique().shape[0])

/tmp/ipykernel_1628383/4267255461.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  merged.groupby("age_group").apply(lambda x: x.ID_REPONDANT.unique().shape[0])
/tmp/ipykernel_1628383/4267255461.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  merged.groupby("age_group").apply(lambda x: x.ID_REPONDANT.unique().shape[0])


age_group
0-11     3019
11-12    1981
13-14       0
15-17       0
>17         0
dtype: int64

In [87]:
merged[['ID_REPONDANT', "ID_ELEVE", "DATE_NAISSANCE", "DATE_TIME", 'age_group']].loc[merged.DATE_TIME.dt.year < 2026].sort_values(["ID_REPONDANT", "DATE_TIME"])


,ID_REPONDANT,ID_ELEVE,DATE_NAISSANCE,DATE_TIME,age_group
12139,E00001,E00001,2014-01-01,2025-02-21,11-12
13435,E00001,E00001,2014-01-01,2025-02-21,11-12
14466,E00001,E00001,2014-01-01,2025-02-21,11-12
17939,E00001,E00001,2014-01-01,2025-02-21,11-12
27774,E00001,E00001,2014-01-01,2025-02-21,11-12
...,...,...,...,...,...
60765,E05000,E05000,2017-01-01,2025-02-20,0-11
79096,E05000,E05000,2017-01-01,2025-02-20,0-11
81700,E05000,E05000,2017-01-01,2025-02-20,0-11
85043,E05000,E05000,2017-01-01,2025-02-20,0-11


In [88]:
merged.shape

(100000, 9)

In [89]:
reponses.shape

(100000, 4)

In [90]:
eleves.ID_ELEVE.unique().shape

(10000,)

In [91]:
reponses.ID_REPONDANT.unique().shape

(5000,)

In [92]:
merged.ID_REPONDANT.unique().shape

(5000,)

In [93]:
merged.ID_ELEVE.unique().shape

(5000,)